1. Dependency and data import


In [ ]:
!pip install tensorflow matplotlib tensorflow-datasets ipywidgets

- configure GPU settings

In [76]:
import tensorflow as tf
# configure the GPUs
gpu_settings = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpu_settings:
  tf.config.experimental.set_memory_growth(gpu, True)


In [ ]:
# should have at least 1 gpu config
gpu_settings

In [78]:
# bring the other dependencies and fashion mnist dataset
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

In [79]:
# download and import our dataset
dataset = tfds.load('fashion_mnist', split='train')

2. Visualize data and build data set


In [80]:
# transform data accordingly
import numpy as np


In [81]:
# set the connection(iterator)
data_iterator = dataset.as_numpy_iterator()

In [ ]:
# get data from pipeline
data_iterator.next()

In [ ]:
# visualize some images from the data set
# set the subplot
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
# get a number of images (4)
for index in range(4):
  # get image and label
  batch = data_iterator.next()
  # plot image on a subplot
  ax[index].imshow(np.squeeze(batch['image']))
  # assign image label as plot title
  ax[index].title.set_text(batch['label'])

In [84]:
# function to scale and return images only
# we do not need the labels, as we will generate undiscriminate outputs
def scale_images(data):
  image = data['image']
  return image/255

In [85]:
# load the dataset anew
dataset = tfds.load('fashion_mnist', split='train')
# run scale_images preprocessing on the dataset
dataset = dataset.map(scale_images)
# chache the dataset batch
dataset = dataset.cache()
# shuffle the dataset
dataset = dataset.shuffle(50000)
# set up batches of 128 images
dataset = dataset.batch(128)
# reduces bottleneck
dataset = dataset.prefetch(64)

In [ ]:
# view configuration of the batch
dataset.as_numpy_iterator().next().shape

3. Build the neural network

We will build two models, a generator and a discriminator. The generator will generate images based on the training data. The discriminator, on the other hand, will try to spot the fakes. Therefore, the models will be trained to do two contradictory tasks.

In [87]:
# get sequential API for the generator and discriminator
from tensorflow.keras.models import Sequential
# get layers for the network
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Reshape, LeakyReLU, Dropout, UpSampling2D

In [88]:
# function to build a generator
def build_generator():
  # initialize model
  model = Sequential()
  # reshapes random value to 7* 7* 28 px
  model.add(Dense(7*7*128, input_dim = 128))
  model.add(LeakyReLU(0.2))
  model.add(Reshape((7,7,128)))

  # upsampling first block
  model.add(UpSampling2D())
  model.add(Conv2D(128, 5, padding='same'))
  model.add(LeakyReLU(0.2))

  # upsampling second block
  model.add(UpSampling2D())
  model.add(Conv2D(128, 5, padding='same'))
  model.add(LeakyReLU(0.2))

  # convolutional first block
  model.add(Conv2D(128, 4, padding='same'))
  model.add(LeakyReLU(0.2))

  # convolutional second block
  model.add(Conv2D(128, 4, padding='same'))
  model.add(LeakyReLU(0.2))

  # final convolutional layer
  model.add(Conv2D(1, 4, padding='same', activation='sigmoid'))

  return model

In [ ]:
# generate the test model
generator = build_generator()

In [ ]:
# view summary
generator.summary()

In [ ]:
# generate new clothes
image = test.predict(np.random.randn(4, 128, 1))
# set the subplot
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
# get a number of images (4)
for index, img in enumerate(image):
  # get image and label
  ax[index].imshow(np.squeeze(img))
  # assign image label as plot title
  ax[index].title.set_text(index)

In [92]:
# build the discriminator
def build_discriminator():
  model = Sequential()

  # convolutional first block
  model.add(Conv2D(32, 5, input_shape=(28,28,1)))
  model.add(LeakyReLU(0.2))
  model.add(Dropout(0.4))

  # convolutional second block
  model.add(Conv2D(64, 5))
  model.add(LeakyReLU(0.2))
  model.add(Dropout(0.4))

  # convolutional third block
  model.add(Conv2D(128, 5))
  model.add(LeakyReLU(0.2))
  model.add(Dropout(0.4))

  # convolutional fourth block
  model.add(Conv2D(256, 5))
  model.add(LeakyReLU(0.2))
  model.add(Dropout(0.4))

  # flatten and pass to dense layer
  model.add(Flatten())
  model.add(Dropout(0.4))
  model.add(Dense(1, activation='sigmoid'))

  return model

In [ ]:
# instantiate a discriminator
discriminator = build_discriminator()

In [ ]:
discriminator.summary()

In [ ]:
# test the discriminator
discriminator.predict(image)

4. Training loops

We will prepare custom training loops to train the generator and discriminator simultaneously.

4.1 Optimizer and loss setup

Ensure that the learning rate for generative is greater than discriminative, in order to ensure that the discriminative does not outlearn generative significantly, swekering the model performance.

In [96]:
# use adam for optimization and binary cross entropy for loss
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

In [97]:
# define generative and discriminative optimization
generation_optimization = Adam(learning_rate = 0.0001)
discrimination_optimization = Adam(learning_rate=0.00001)
# define generative and discriminative losses
generation_loss = BinaryCrossentropy()
discrimination_loss = BinaryCrossentropy()

4.2 Subclass model building


In [98]:
# import the base model class for our training step
from tensorflow.keras.models import Model

In [99]:
# define the governing class for the training
class fashionista(Model):
  def __init__(self, generator, discriminator, *args, **kwargs):
    # pass to base class
    super().__init__(*args, **kwargs)

    # create attributes for generator and discriminator
    self.generator = generator
    self.discriminator = discriminator

  def compile(self, gopt, dopt, gloss, dloss, *args, **kwargs):
    # compile with base class
    super().compile(*args, **kwargs)

    # attributes for optimization and losses
    self.gopt = gopt
    self.dopt = dopt
    self.gloss = gloss
    self.dloss = dloss

  def train_step(self, batch):
    # get the batch of data (128 in our case)
    real_images = batch
    fake_images = self.generator(tf.random.normal((128, 128, 1)), training=False)

    # train the discriminator
    with tf.GradientTape() as d_tape:

      # pass real and fake imnages to discriminator
      y_hat_real = self.discriminator(real_images, training=True)
      y_hat_fake = self.discriminator(fake_images, training=True)
      y_hat_real_fake = tf.concat([y_hat_real, y_hat_fake], axis=0)

      # label the images as real and fake
      y_real_fake = tf.concat([tf.zeros_like(y_hat_real), tf.ones_like(y_hat_fake)], axis=0)

      # add noise to the output to avoid outlearning
      noise_real = 0.15*tf.random.uniform(tf.shape(y_hat_real))
      noise_fake = -0.15*tf.random.uniform(tf.shape(y_hat_fake))
      y_real_fake += tf.concat([noise_real, noise_fake], axis=0)

      # calculate loss
      total_dis_loss = self.dloss(y_real_fake, y_hat_real_fake)

    # apply learning with backpropagation method
    d_gradient = d_tape.gradient(total_dis_loss, self.discriminator.trainable_variables)
    self.dopt.apply_gradients(zip(d_gradient, self.discriminator.trainable_variables))

    with tf.GradientTape() as g_tape:
      # generate new images
      gen_images = self.generator(tf.random.normal((128, 128, 1)), training=True)

      # create predicted labels
      predicted_labels = self.discriminator(gen_images, training=False)

      # calculate loss
      total_gen_loss = self.gloss(tf.zeros_like(predicted_labels), predicted_labels)

    # apply backpropagation
    ggrad = g_tape.gradient(total_gen_loss, self.generator.trainable_variables)
    self.gopt.apply_gradients(zip(ggrad, self.generator.trainable_variables))

    return {"dis_loss": total_dis_loss, "gen_loss": total_gen_loss}

In [100]:
# create instance of the model
fashioner = fashionista(generator=test, discriminator=discriminator)

In [101]:
# compile the model
fashioner.compile(gopt=generation_optimization, dopt=discrimination_optimization, gloss=generation_loss, dloss=discrimination_loss)

4.3 Callback Building


In [102]:
# import the dependencies
import os
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.callbacks import Callback

In [103]:
class ModelMonitor(Callback):
  def __init__(self, num_img=3, latent_dim=128):
    self.num_img = num_img
    self.latent_dim = latent_dim
  def on_epoch_end(self, epoch, legs=None):
    random_latent_vectors = tf.random.uniform((self.num_img, self.latent_dim, 1))
    generated_images = self.model.generator(random_latent_vectors)
    generated_images *= 255
    generated_images.numpy()
    for i in range(self.num_img):
      img = array_to_img(generated_images[i])
      img.save(os.path.join('images', f'generated_image_{epoch}_{i}.png'))

4.3 Train

In [ ]:
dataset.as_numpy_iterator().next().shape

In [ ]:
# run the loop
history = fashioner.fit(dataset, epochs=20, callbacks=[ModelMonitor()])

4.4 Performance Review

In [ ]:
# use history and matplotlib for loss visualization
plt.suptitle('Overall loss')
plt.plot(history.history['dis_loss'], label='Discriminator loss')
plt.plot(history.history['gen_loss'], label='Generator loss')
plt.legend()
plt.show()

5. Generator testing

In [ ]:
# generate images
# use pre_wieghted tests for faster results
generator.load_weights(os.path.join('archive', 'generatormodel.h5'))

images = generator.predict(tf.random.normal((16, 128, 1)))

In [ ]:
# use the images to generate fashionable items images
fig, ax = plt.subplots(ncols=4, nrows=4, figsize=(20, 20))
for x in range(4):
  for y in range(4):
    ax[x][y].imshow(images[(x+1)* (y+1)-1])

In [ ]:
# save the model

generator.save('generator_dk.h5')
discriminator.save('discriminator.h5')